In [1]:
from langchain.document_loaders import SeleniumURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain import LLMChain

In [2]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

# 1: Load the data from urls
loader = SeleniumURLLoader(urls=urls)
entire_data = loader.load()

# 2: Split them into chunks using the CharacterTextSplitter with a chunk size of 1000 and no overlap:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
data = text_splitter.split_documents(entire_data)


In [3]:
# 3: Compute the embeddings using OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
# 4: store them in a Deep Lake vector store on the cloud
my_activeloop_org_id = "srishtysuman2919" 
customer_support_chatbot = "customer_support_chatbot"
dataset_path = f"https:/app.activeloop.ai/srishtysuman2919/{customer_support_chatbot}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(data)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.
Creating 65 embeddings in 1 batches of size 65:: 100%|██████████| 1/1 [00:05<00:00,  5.20s/it]

Dataset(path='https:/app.activeloop.ai/srishtysuman2919/customer_support_chatbot', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (65, 1)      str     None   
 metadata     json      (65, 1)      str     None   
 embedding  embedding  (65, 1536)  float32   None   
    id        text      (65, 1)      str     None   


['1454c308-c6b0-11ee-ba01-acde48001122',
 '1454c3f8-c6b0-11ee-ba01-acde48001122',
 '1454c48e-c6b0-11ee-ba01-acde48001122',
 '1454c51a-c6b0-11ee-ba01-acde48001122',
 '1454c556-c6b0-11ee-ba01-acde48001122',
 '1454c592-c6b0-11ee-ba01-acde48001122',
 '1454c5ba-c6b0-11ee-ba01-acde48001122',
 '1454c628-c6b0-11ee-ba01-acde48001122',
 '1454c664-c6b0-11ee-ba01-acde48001122',
 '1454c68c-c6b0-11ee-ba01-acde48001122',
 '1454c6be-c6b0-11ee-ba01-acde48001122',
 '1454c6e6-c6b0-11ee-ba01-acde48001122',
 '1454c70e-c6b0-11ee-ba01-acde48001122',
 '1454c736-c6b0-11ee-ba01-acde48001122',
 '1454c75e-c6b0-11ee-ba01-acde48001122',
 '1454c786-c6b0-11ee-ba01-acde48001122',
 '1454c7ae-c6b0-11ee-ba01-acde48001122',
 '1454c7d6-c6b0-11ee-ba01-acde48001122',
 '1454c7fe-c6b0-11ee-ba01-acde48001122',
 '1454c826-c6b0-11ee-ba01-acde48001122',
 '1454c844-c6b0-11ee-ba01-acde48001122',
 '1454c86c-c6b0-11ee-ba01-acde48001122',
 '1454c894-c6b0-11ee-ba01-acde48001122',
 '1454c8bc-c6b0-11ee-ba01-acde48001122',
 '1454c8e4-c6b0-

In [5]:
# 5: To retrieve the most similar chunks to a given query, we can use the similarity_search method of the Deep Lake vector store
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

# 6: Craft a prompt for GPT-3-> create a prompt template that incorporates role-prompting, relevant Knowledge Base information, and the user's question:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

# 7: Utilize the GPT3 model with a temperature of 0 for text generation
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# user question
query = "How to check disk usage in linux?"

# 8: retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# 9: format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# 10: generate answer
answer = llm(prompt_formatted)
print(answer)

Check Disk Usage Using Gnome Disk Tool

Check Disk Usage Using Disk Usage Analyzer Tool

Cleanup Disk using Disk Usage Analyzer

Check Disk Space Using the df Command

In Linux, there are many commands to check disk usage, the most common being the df command. The df stands for “Disk Filesystem” in the command, which is a handy way to check the current disk usage and the available disk space in Linux. The syntax for the df command in Linux is as follows:

df <options> <file_system>

The options to use with the df command are:

a

Show information about all file systems including pseudo, duplicate and inaccessible file systems

h

Display the sizes in human-readable format i.e in powers of 1024

t

Display the disk usage of only the file system of a particular type

x

Display the disk usage excluding a particular file type

Display Disk Usage in Human Readable Format


/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 There are several ways to check disk usage in Linux. One of the most common methods is by using the df command. You can use the options "a" to show information about all file systems, "h" to display sizes in human-readable format, "t" to display disk usage of a particular file system type, or "x" to exclude a specific file type. Another way is by using GUI tools such as the Gnome Disk Tool or the Disk Usage Analyzer Tool. These tools provide a visual representation of disk usage and allow you to easily identify and delete large files or folders.
